In [ ]:
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

from os import getenv
from dotenv import load_dotenv
import requests, random

In [ ]:
load_dotenv()

In [ ]:
#Fetch Store all free models
models=[]
free_models = []
model_req_uri = getenv("OPENROUTER_URI")+"/models"
response = requests.get(model_req_uri)
for model in response.json()['data']:
    if model['pricing']['prompt'] == '0' and model['pricing']['completion'] == '0' and model['pricing']['image'] == '0':
        free_models.append({
            'id': model['id'],
            'name': model['name'],
            'context_length': model['context_length'],
            'modality': model['architecture']['modality'],
            'tokenizer': model['architecture']['tokenizer'],
            'pricing': model['pricing'],
            'max_completion_tokens': model['top_provider']['max_completion_tokens'],
            'moderated': model['top_provider']['is_moderated'],
            'request_limit': model['per_request_limits']
            })
    else:
        models.append({
            'id': model['id'],
            'name': model['name'],
            'context_length': model['context_length'],
            'modality': model['architecture']['modality'],
            'tokenizer': model['architecture']['tokenizer'],
            'pricing': model['pricing'],
            'max_completion_tokens': model['top_provider']['max_completion_tokens'],
            'moderated': model['top_provider']['is_moderated'],
            'request_limit': model['per_request_limits']
            })

models.extend(free_models)

In [ ]:
text_models = [m for m in free_models if 'image' not in m['modality']]

random_text_model = random.choice(text_models)
print(random_text_model)

In [ ]:
if model:
    prompt = ChatPromptTemplate.from_template("tell me a joke about {topic}. 200 words")
    parser = StrOutputParser()
    llm = ChatOpenAI(
    api_key=getenv("OPENROUTER_TOKEN"),
    base_url=getenv("OPENROUTER_URI"),
    model=random_text_model['id'],
    tiktoken_model_name=random_text_model['tokenizer'],
    max_completion_tokens=random_text_model['max_completion_tokens'],
    temperature=0.1,
    streaming=True,
    )

    chain = prompt | llm | parser

In [ ]:
if chain:
    print(f"Model Used: {random_text_model['name']}")
    async for chunk in chain.astream({"topic": "parrot"}):
        print(chunk, end="", flush=True)